<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import sys
sys.path.append('../src')

import numpy as np
import utils_circuit
from importlib import reload
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import DR_RBF
import KES
import SES
import sklearn_transformers

size = 22

import matplotlib.pylab as pylab
params = {'legend.fontsize': size,
          'figure.figsize': (7, 5),
          'axes.labelsize': size,
          'axes.titlesize': size,
          'xtick.labelsize':size,
          'ytick.labelsize':size}
pylab.rcParams.update(params)

**Fixed parameters for the experiment**

In [60]:
N_bags=50
N_items=20
nb_periods=20
spec_param={'C':[1.,1.],'phi':[np.pi/8,np.pi/2],'em':[1,1],'omega':[5,5]}

**What we will vary during the experiment**

In [61]:
subsample = [25,24,23,21,18,15,12,9,6] 

**To store the results**

In [69]:
df_RBF_m = pd.DataFrame(index=['mean_RBF_rmse'], columns=subsample)
df_RBF_std = pd.DataFrame(index=['std_RBF_rmse'], columns=subsample)
df_KES_m = pd.DataFrame(index=['mean_KES_rmse'], columns=subsample)
df_KES_std = pd.DataFrame(index=['std_KES_rmse'], columns=subsample)
df_SES_m = pd.DataFrame(index=['mean_SES_rmse'], columns=subsample)
df_SES_std = pd.DataFrame(index=['std_SES_mse'], columns=subsample)

**Run the experiment**

In [4]:
cv=3
reload(utils_circuit)
for j,param in enumerate(subsample):
    
    print(j)
        
    ''' GENERATE DATA '''
    t_span = np.linspace(0, nb_periods * 2 * np.pi / spec_param['omega'][0], nb_periods * 25)
    
    # this is what changes
    nb_obs = nb_periods * param
    
    # generate new dataset
    example = utils_circuit.Circuit(N_bags=N_bags, N_items=N_items, spec_param=spec_param, t_span=t_span, nb_obs=nb_obs)
    example.generate_data()
    example.get_phi()
    
    X = example.paths
    y = example.labels[:,0]
    
    # SES model  
    mean,std,_  = SES.model(depths1=[2], depth2=2, X=X, y=y, ll=None, at=False, NUM_TRIALS=5, cv=cv)
    df_SES_m.iloc[0,j] = mean
    df_SES_std.iloc[0,j] = std   
    
    # KES model
    mean,std,_  = KES.model(X=X, y=y, ll=None, at=False, scales=[0.03], NUM_TRIALS=5, cv=cv)
    df_KES_m.iloc[0,j] = mean
    df_KES_std.iloc[0,j] = std
    
    # DR-RBF model
    mean,std,_ = DR_RBF.model(X=X, y=y, NUM_TRIALS=5,cv=cv)
    df_RBF_m.iloc[0,j] = mean
    df_RBF_std.iloc[0,j] = std
   

In [71]:
m = pd.concat([df_RBF_m,df_SES_m,df_KES_m],join="inner")
stdv = pd.concat([df_RBF_std,df_SES_std,df_KES_std],join="inner")

In [5]:
fig,ax = plt.subplots(1,figsize=(10,7))

labels = ['DR-RBF','SES','KES']
colors = ['red','blue','green']
markers = ['o','^', 'p']
linestyles = ['dotted','solid', 'dashed']

for i in [0,1,2]:

    ax.plot([100-100*e/25 for e in subsample],m.iloc[i].ewm(alpha=0.1).mean().values.astype(float),color=colors[i],
            label=labels[i],marker=markers[i],fillstyle='none',lw=3,markersize=10,linestyle=linestyles[i])
    fact=1
    
    params = np.array([100-100*e/25 for e in subsample])
    plus = m.iloc[i].ewm(alpha=0.1).mean().values.astype(float)  + fact*stdv.iloc[i].ewm(alpha=0.1).mean().values.astype(float)
    minus = m.iloc[i].ewm(alpha=0.1).mean().values.astype(float) - fact*stdv.iloc[i].ewm(alpha=0.1).mean().values.astype(float).astype(float)

    ax.fill_between(params, minus,plus,alpha=0.2,color=colors[i])


legend = ax.legend(loc='upper left')
ax.set_ylabel(r'Predictive MSE',labelpad=10)
#ax.yaxis.set_ticks(np.arange(0.05, 0.50, 0.05))
ax.set_xlabel(r'Subsampling rate (%)',labelpad=10)
ax.grid(alpha=0.9, linestyle='dashed', linewidth=0.5)


# plt.savefig('../pictures/circuit.pdf')
plt.show()